In [7]:
import numpy as np
import os
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.models.resnet import ResNet, BasicBlock, Bottleneck
import torch.nn.functional as F
import pandas as pd
import torch.utils.data as data
import argparse
import sys

In [8]:
class AdaptedResNet(ResNet):
    def __init__(self):
        super(AdaptedResNet, self).__init__(BasicBlock, [2, 2, 2, 2], num_classes=2) # Based on ResNet18
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=1, padding=3,bias=False)

class GenreClassifier(nn.Module):
    def __init__(self):
        super(GenreClassifier, self).__init__()

        self.fc = nn.Linear(2998, 3 * 32 * 32)
        self.net = AdaptedResNet()
        
    def forward(self, x):
        x = self.fc(x)
        x = x.view(-1, 3, 32, 32) 
        x = self.net(x)
        return x

In [9]:
def train(args, model, device, train_loader, optimizer, epoch, loss_fn):
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tlr: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), optimizer.param_groups[0]['lr']))
            if args.dry_run:
                break

In [17]:
def test(model, device, test_loader, loss_fn):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    acc = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        acc))
    return acc

In [18]:
class BeatDataset(data.Dataset):
    def __init__(self, train):
        if train:
            self.dataset = np.load('train_data.npy')
            self.labels = np.load('train_labels.npy')
        else:
            self.dataset = np.load('test_data.npy')
            self.labels = np.load('test_labels.npy')
            
        self.labels = np.array([(0 if l=='Jazz' else 1) for l in self.labels]).astype('int64')
#         self.transform_fn = transform=transforms.Compose([
#                                 transforms.ToTensor(),
#                                 #transforms.Normalize((0.5,), (0.5,))
#                                 ])
    def __getitem__(self, index):
        data = torch.from_numpy(self.dataset[index])
        label = torch.tensor(self.labels[index])
        return (data, label)
    def __len__(self):
        return len(self.labels)

In [21]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument("-f", "--fff", help="dummy argument", default="1")
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=200, metavar='N',
                    help='number of epochs to train (default: 14)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--dry-run', action='store_true', default=False,
                    help='quickly check a single pass')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
parser.add_argument('--save-model', type=str, default='first')
args = parser.parse_args(sys.argv[1:])
use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': args.batch_size, 'shuffle': True}
test_kwargs = {'batch_size': args.test_batch_size, 'shuffle': False}
if use_cuda:
    cuda_kwargs = {'num_workers': 0,
                   'pin_memory': False}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

trainset = BeatDataset(train=True)
testset = BeatDataset(train=False)

train_loader = torch.utils.data.DataLoader(trainset,**train_kwargs)
test_loader = torch.utils.data.DataLoader(testset, **test_kwargs)

loss_fn = nn.CrossEntropyLoss()

model = GenreClassifier().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, betas=(0.5, 0.999), weight_decay=5e-4)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.99)
best_acc = 0
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch, loss_fn)
    acc = test(model, device, test_loader, loss_fn)
    scheduler.step()

    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), '{}.pt'.format(args.save_model))
        print('Saved model to {}.pt'.format(args.save_model))
print('Best Accuracy: {}'.format(best_acc))

Train Epoch: 1 [0/1012 (0%)]	Loss: 1.001123	lr: 0.000010
Train Epoch: 1 [640/1012 (62%)]	Loss: 0.736548	lr: 0.000010

Test set: Average loss: 0.0048, Accuracy: 85/179 (47.49%)

Saved model to first.pt
Train Epoch: 2 [0/1012 (0%)]	Loss: 0.539537	lr: 0.000010
Train Epoch: 2 [640/1012 (62%)]	Loss: 0.475308	lr: 0.000010

Test set: Average loss: 0.0066, Accuracy: 85/179 (47.49%)

Train Epoch: 3 [0/1012 (0%)]	Loss: 0.297349	lr: 0.000010
Train Epoch: 3 [640/1012 (62%)]	Loss: 0.237899	lr: 0.000010

Test set: Average loss: 0.0082, Accuracy: 85/179 (47.49%)

Train Epoch: 4 [0/1012 (0%)]	Loss: 0.218953	lr: 0.000010
Train Epoch: 4 [640/1012 (62%)]	Loss: 0.201059	lr: 0.000010

Test set: Average loss: 0.0060, Accuracy: 88/179 (49.16%)

Saved model to first.pt
Train Epoch: 5 [0/1012 (0%)]	Loss: 0.190221	lr: 0.000010
Train Epoch: 5 [640/1012 (62%)]	Loss: 0.158917	lr: 0.000010

Test set: Average loss: 0.0036, Accuracy: 118/179 (65.92%)

Saved model to first.pt
Train Epoch: 6 [0/1012 (0%)]	Loss: 0.08554

Train Epoch: 46 [640/1012 (62%)]	Loss: 0.001437	lr: 0.000006

Test set: Average loss: 0.0039, Accuracy: 131/179 (73.18%)

Train Epoch: 47 [0/1012 (0%)]	Loss: 0.001843	lr: 0.000006
Train Epoch: 47 [640/1012 (62%)]	Loss: 0.001282	lr: 0.000006

Test set: Average loss: 0.0039, Accuracy: 131/179 (73.18%)

Train Epoch: 48 [0/1012 (0%)]	Loss: 0.001001	lr: 0.000006
Train Epoch: 48 [640/1012 (62%)]	Loss: 0.001635	lr: 0.000006

Test set: Average loss: 0.0039, Accuracy: 131/179 (73.18%)

Train Epoch: 49 [0/1012 (0%)]	Loss: 0.001112	lr: 0.000006
Train Epoch: 49 [640/1012 (62%)]	Loss: 0.001812	lr: 0.000006

Test set: Average loss: 0.0040, Accuracy: 131/179 (73.18%)

Train Epoch: 50 [0/1012 (0%)]	Loss: 0.000966	lr: 0.000006
Train Epoch: 50 [640/1012 (62%)]	Loss: 0.001589	lr: 0.000006

Test set: Average loss: 0.0039, Accuracy: 131/179 (73.18%)

Train Epoch: 51 [0/1012 (0%)]	Loss: 0.000984	lr: 0.000006
Train Epoch: 51 [640/1012 (62%)]	Loss: 0.001458	lr: 0.000006

Test set: Average loss: 0.0038, Accura


Test set: Average loss: 0.0042, Accuracy: 132/179 (73.74%)

Train Epoch: 92 [0/1012 (0%)]	Loss: 0.000564	lr: 0.000004
Train Epoch: 92 [640/1012 (62%)]	Loss: 0.000294	lr: 0.000004

Test set: Average loss: 0.0041, Accuracy: 132/179 (73.74%)

Train Epoch: 93 [0/1012 (0%)]	Loss: 0.000302	lr: 0.000004
Train Epoch: 93 [640/1012 (62%)]	Loss: 0.000348	lr: 0.000004

Test set: Average loss: 0.0042, Accuracy: 132/179 (73.74%)

Train Epoch: 94 [0/1012 (0%)]	Loss: 0.000599	lr: 0.000004
Train Epoch: 94 [640/1012 (62%)]	Loss: 0.000560	lr: 0.000004

Test set: Average loss: 0.0042, Accuracy: 131/179 (73.18%)

Train Epoch: 95 [0/1012 (0%)]	Loss: 0.000337	lr: 0.000004
Train Epoch: 95 [640/1012 (62%)]	Loss: 0.000303	lr: 0.000004

Test set: Average loss: 0.0042, Accuracy: 132/179 (73.74%)

Train Epoch: 96 [0/1012 (0%)]	Loss: 0.000362	lr: 0.000004
Train Epoch: 96 [640/1012 (62%)]	Loss: 0.000273	lr: 0.000004

Test set: Average loss: 0.0041, Accuracy: 132/179 (73.74%)

Train Epoch: 97 [0/1012 (0%)]	Loss: 0.0

Train Epoch: 137 [640/1012 (62%)]	Loss: 0.000661	lr: 0.000003

Test set: Average loss: 0.0044, Accuracy: 131/179 (73.18%)

Train Epoch: 138 [0/1012 (0%)]	Loss: 0.000186	lr: 0.000003
Train Epoch: 138 [640/1012 (62%)]	Loss: 0.000287	lr: 0.000003

Test set: Average loss: 0.0044, Accuracy: 132/179 (73.74%)

Train Epoch: 139 [0/1012 (0%)]	Loss: 0.000206	lr: 0.000002
Train Epoch: 139 [640/1012 (62%)]	Loss: 0.000354	lr: 0.000002

Test set: Average loss: 0.0043, Accuracy: 133/179 (74.30%)

Train Epoch: 140 [0/1012 (0%)]	Loss: 0.000424	lr: 0.000002
Train Epoch: 140 [640/1012 (62%)]	Loss: 0.000198	lr: 0.000002

Test set: Average loss: 0.0043, Accuracy: 133/179 (74.30%)

Train Epoch: 141 [0/1012 (0%)]	Loss: 0.000205	lr: 0.000002
Train Epoch: 141 [640/1012 (62%)]	Loss: 0.000443	lr: 0.000002

Test set: Average loss: 0.0042, Accuracy: 134/179 (74.86%)

Train Epoch: 142 [0/1012 (0%)]	Loss: 0.000197	lr: 0.000002
Train Epoch: 142 [640/1012 (62%)]	Loss: 0.000293	lr: 0.000002

Test set: Average loss: 0.0


Test set: Average loss: 0.0044, Accuracy: 132/179 (73.74%)

Train Epoch: 183 [0/1012 (0%)]	Loss: 0.000179	lr: 0.000002
Train Epoch: 183 [640/1012 (62%)]	Loss: 0.000215	lr: 0.000002

Test set: Average loss: 0.0044, Accuracy: 133/179 (74.30%)

Train Epoch: 184 [0/1012 (0%)]	Loss: 0.000134	lr: 0.000002
Train Epoch: 184 [640/1012 (62%)]	Loss: 0.000275	lr: 0.000002

Test set: Average loss: 0.0045, Accuracy: 132/179 (73.74%)

Train Epoch: 185 [0/1012 (0%)]	Loss: 0.000200	lr: 0.000002
Train Epoch: 185 [640/1012 (62%)]	Loss: 0.000332	lr: 0.000002

Test set: Average loss: 0.0045, Accuracy: 133/179 (74.30%)

Train Epoch: 186 [0/1012 (0%)]	Loss: 0.000168	lr: 0.000002
Train Epoch: 186 [640/1012 (62%)]	Loss: 0.000136	lr: 0.000002

Test set: Average loss: 0.0044, Accuracy: 133/179 (74.30%)

Train Epoch: 187 [0/1012 (0%)]	Loss: 0.000189	lr: 0.000002
Train Epoch: 187 [640/1012 (62%)]	Loss: 0.000292	lr: 0.000002

Test set: Average loss: 0.0044, Accuracy: 133/179 (74.30%)

Train Epoch: 188 [0/1012 (0%)